# Bloch-Torrey Equation

## Introduction

Here we solve the Bloch-Torrey equation on a unit square, with the diffusion coefficient $D(x)$, relaxation rate $R(x)$, and resonance frequency $\omega(x)$ all given as a generic functions.
The strong form of the Bloch-Torrey equation is given by

\begin{align}
    \frac{\partial u_x}{\partial t} &= \nabla \cdot (D \nabla u_x) - R u_x + \omega u_y  \quad x \in \Omega\\
    \frac{\partial u_y}{\partial t} &= \nabla \cdot (D \nabla u_y) - R u_y - \omega u_x  \quad x \in \Omega,
\end{align}

where $\vec{u}=[u_x,u_y]$ is the transverse magnetization, and $\Omega$ the domain.

We will consider homogeneous Neumann boundary conditions such that

\begin{align}
    \nabla \vec{u}(x) \cdot \hat{n} &= 0  \quad x \in \partial \Omega\\
\end{align}

where $\partial \Omega$ denotes the boundary of $\Omega$. The initial condition is given generically as

\begin{equation}
    \vec{u}(x,t=0) = \vec{u}_0 (x)  \quad x \in \Omega
\end{equation}


The resulting weak form is given by
\begin{align}
    \int_{\Omega} \vec{v} \cdot \vec{u}_t \, d\Omega
    &= -\int_{\Omega}
    -\vec{v} \cdot \nabla \cdot ( D \, \nabla \vec{u} ) +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega \\
    &= -\int_{\Omega}
    D \, \nabla \vec{v} : \nabla \vec{u} +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega + 
    \int_{\partial\Omega} \vec{v} \cdot (D\nabla\vec{u} \cdot \hat{n}) \, d\Gamma,
\end{align}
where $\vec{v}$ is a suitable test function.

In this notebook, we will assume homogeneous Neumann boundary conditions on all boundaries by taking $D\nabla\vec{u} \cdot \hat{n} = 0$. Therefore, the final weak form is simply
\begin{align}
    \int_{\Omega} \vec{v} \cdot \vec{u}_t \, d\Omega
    = -\int_{\Omega}
    D \, \nabla \vec{v} : \nabla \vec{u} +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega
\end{align}

Note that, in two dimensions, the cross product is simply a scalar. However, `Tensors.jl` defines the two dimensional cross product by first extending the 2D vectors into 3D. Below, we use the symbol $\boxtimes$ to denote the scalar version, which is the same as taking the third component of the vector version

## Commented Program

Now we solve the problem in JuAFEM. What follows is a program spliced with comments.

First we load the JuAFEM package.

In [6]:
#HOME = "/home/jon/Documents/UBCMRI/"
HOME = "/home/coopar7/Documents/code/"
cd(HOME * "BlochTorreyExperiments-master/")
include("Experiments/MyelinWaterOrientation/init.jl")

**Pack circles**: Pack circles with a specified packing density $\eta$

In [7]:
btparams = BlochTorreyParameters();
freqparams = FreqMapParams(btparams);

In [8]:
# α or k == R_shape, θ == R_scale
R_mu = 0.46 # Axon mean radius [um] ; this is taken to be outer radius
R_shape = 5.7 # Axon radius shape parameter for Gamma distribution (Xu)
R_scale = R_mu / R_shape # Axon radius scale parameter [um]
R_σ = sqrt(R_shape)*R_scale; # Axon radius variance

const Dim = 2
Ncircles = 50
rs = rand(Gamma(R_shape, R_scale), Ncircles);
os = initialize_origins(rs);

η = 0.7 # goal packing density
ϵ = 0.1*R_mu # overlap occurs when distance between circle edges is ≤ ϵ
α = 0.0 # density penalty weight
β = 1e-5 # mutual distance penalty weight
λ = 1.0 # overlap penalty weight (or lagrange multiplier for constrained version)
w = [α, β, λ]; # vector of weights

In [9]:
revise()

In [10]:
@time outer_circles, opt_result = pack_circles(rs;
    autodiff = true,
    reversemode = false,
    secondorder = false,
    setcallback = true,
    initial_origins = os,
    goaldensity = η,
    distancescale = R_mu,
    weights = w,
    epsilon = ϵ);

In [11]:
opt_result

Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [-24.279801734509775,-10.36065371766216, ...]
 * Minimizer: [-8.126783921510087,-0.4487420372615113, ...]
 * Minimum: 5.106707e-01
 * Iterations: 703
 * Convergence: false
   * |x - x'| ≤ 1.0e-32: false 
     |x - x'| = 1.79e-01 
   * |f(x) - f(x')| ≤ 1.0e-32 |f(x)|: false
     |f(x) - f(x')| = 2.74e-03 |f(x)|
   * |g(x)| ≤ 1.0e-12: false 
     |g(x)| = 5.64e-02 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 838
 * Gradient Calls: 704

 19.469397 seconds (17.17 M allocations: 1.049 GiB, 5.66% gc time)


In [12]:
estimate_density(outer_circles)

0.7

In [13]:
is_any_overlapping(outer_circles)

false

In [14]:
@show (minimum_signed_edge_distance(outer_circles), ϵ);

In [15]:
revise()

In [16]:
g_ratio = 0.5 #0.8370
#rect_bdry = bounding_box(outer_circles)
rect_bdry = scale_shape(inscribed_square(crude_bounding_circle(outer_circles)), 0.75)
#rect_bdry = scale_shape(bounding_box(outer_circles), 1.05)

outer_circles = filter(c -> !is_outside(c, rect_bdry), outer_circles)
inner_circles = scale_shape.(outer_circles, g_ratio);
Ncircles = length(outer_circles)

48

In [99]:
revise()

In [100]:
#Nmin = 50; # points for average circle
#h0 = 2pi*R_mu/Nmin; # approximate scale
h0 = 0.3*R_mu*(1-g_ratio) # fraction of size of average torus width
eta = 3.0; # approx ratio between largest/smallest edges

# @time grid = rect_mesh_with_tori(rect_bdry, inner_circles, outer_circles, h0, eta, isunion=true);
# @time grid, subgrids = rect_mesh_with_circles(rect_bdry, outer_circles, h0, eta, isunion=false);

In [101]:
#mxcall(:figure,0); mxcall(:hold,0,"on"); mxplot(grid); sleep(0.1);

In [105]:
exteriorgrid, torigrids, interiorgrids = disjoint_rect_mesh_with_tori(rect_bdry, inner_circles, outer_circles, h0, eta);
#@time exteriorgrid, torigrids, interiorgrids = rect_mesh_with_circles(rect_bdry, outer_circles, h0, eta, isunion=false);

0/48: Exterior
1/48: Interior
>> Warning: qhull precision warning:
The initial hull is narrow (cosine of min. angle is 1.0000000000000002).
A coplanar point may lead to a wide facet.  Options 'QbB' (scale to unit box)
or 'Qbb' (scale last coordinate) may remove this warning.  Use 'Pp' to skip
this warning.  See 'Limitations' in qh-impre.htm.
 
The initial hull is narrow (cosine of min. angle is 1.0000000000000000).
A coplanar point may lead to a wide facet.  Options 'QbB' (scale to unit box)
or 'Qbb' (scale last coordinate) may remove this warning.  Use 'Pp' to skip
this warning.  See 'Limitations' in qh-impre.htm.
 
The initial hull is narrow (cosine of min. angle is 1.0000000000000000).
A coplanar point may lead to a wide facet.  Options 'QbB' (scale to unit box)
or 'Qbb' (scale last coordinate) may remove this warning.  Use 'Pp' to skip
this warning.  See 'Limitations' in qh-impre.htm.
 
The initial hull is narrow (cosine of min. angle is 0.9999999999999999).
A coplanar point may le

7/48: Annular
8/48: Interior
8/48: Annular
>> Warning: qhull precision warning:
The initial hull is narrow (cosine of min. angle is 1.0000000000000000).
A coplanar point may lead to a wide facet.  Options 'QbB' (scale to unit box)
or 'Qbb' (scale last coordinate) may remove this warning.  Use 'Pp' to skip
this warning.  See 'Limitations' in qh-impre.htm.
 
The initial hull is narrow (cosine of min. angle is 1.0000000000000000).
A coplanar point may lead to a wide facet.  Options 'QbB' (scale to unit box)
or 'Qbb' (scale last coordinate) may remove this warning.  Use 'Pp' to skip
this warning.  See 'Limitations' in qh-impre.htm.
 
The initial hull is narrow (cosine of min. angle is 1.0000000000000000).
A coplanar point may lead to a wide facet.  Options 'QbB' (scale to unit box)
or 'Qbb' (scale last coordinate) may remove this warning.  Use 'Pp' to skip
this warning.  See 'Limitations' in qh-impre.htm.
 
The initial hull is narrow (cosine of min. angle is 1.0000000000000000).
A coplanar

9/48: Interior
9/48: Annular
10/48: Interior
10/48: Annular
11/48: Interior
11/48: Annular
12/48: Interior
12/48: Annular
>> Warning: qhull precision warning:
The initial hull is narrow (cosine of min. angle is 1.0000000000000000).
A coplanar point may lead to a wide facet.  Options 'QbB' (scale to unit box)
or 'Qbb' (scale last coordinate) may remove this warning.  Use 'Pp' to skip
this warning.  See 'Limitations' in qh-impre.htm.
 
13/48: Interior
13/48: Annular
14/48: Interior
14/48: Annular


LoadError: [91mInterruptException:[39m

15/48: Interior


In [ ]:
#all_tori = form_subgrid(grid, getcellset(grid, "tori"), getnodeset(grid, "tori"), getfaceset(grid, "boundary"))
#all_int = form_subgrid(grid, getcellset(grid, "interior"), getnodeset(grid, "interior"), getfaceset(grid, "boundary"))

In [ ]:
mxcall(:figure,0); mxcall(:hold,0,"on"); mxplot(exteriorgrid); sleep(0.1);
mxcall(:figure,0); mxcall(:hold,0,"on"); mxplot(all_tori); sleep(0.1);
mxcall(:figure,0); mxcall(:hold,0,"on"); mxplot(all_int); sleep(0.1);

In [ ]:
domain = MyelinDomain(grid, outer_circles, inner_circles, rect_bdry, exteriorgrid, torigrids, interiorgrids;
    quadorder = 1, funcinterporder = 1);

In [ ]:
doassemble!(domain, btparams);

In [ ]:
factorize!(domain);

In [ ]:
using Cubature

In [ ]:
lb, ub = Vector(minimum(rect_bdry)), Vector(maximum(rect_bdry))
(val,err) = hcubature(1, (x,v) -> (y = exp.(2*x.^2/10); v[1] = dot(y,y)), lb, ub)

In [ ]:
# U = interpolate(x->Vec{2}(exp.(2x⊙x/10)), domain);
U = interpolate(Vec{2}((0.0,1.0)), domain);
S = integrate(U, domain)

In [ ]:
norm(U, domain)

### Diffusion coefficient $D(x)$, relaxation rate $R(x)$, and resonance frequency $\omega(x)$

These functions are defined within `doassemble!`

### Trial and test functions
A `CellValues` facilitates the process of evaluating values and gradients of
test and trial functions (among other things). Since the problem
is a scalar problem we will use a `CellScalarValues` object. To define
this we need to specify an interpolation space for the shape functions.
We use Lagrange functions (both for interpolating the function and the geometry)
based on the reference "cube". We also define a quadrature rule based on the
same reference cube. We combine the interpolation and the quadrature rule
to a `CellScalarValues` object.

### Degrees of freedom
Next we need to define a `DofHandler`, which will take care of numbering
and distribution of degrees of freedom for our approximated fields.
We create the `DofHandler` and then add a single field called `u`.
Lastly we `close!` the `DofHandler`, it is now that the dofs are distributed
for all the elements.

Now that we have distributed all our dofs we can create our tangent matrix,
using `create_sparsity_pattern`. This function returns a sparse matrix
with the correct elements stored.

We can inspect the pattern using the `spy` function from `UnicodePlots.jl`.
By default the stored values are set to $0$, so we first need to
fill the stored values, e.g. `K.nzval` with something meaningful.

In [ ]:
#using UnicodePlots
#fill!(K.nzval, 1.0)
#spy(K; height = 25)

### Boundary conditions
In JuAFEM constraints like Dirichlet boundary conditions are handled by a `ConstraintHandler`. However, here we will have no need to directly enforce boundary conditions, since Neumann boundary conditions have already been applied in the derivation of the weak form.

### Assembling the linear system
Now we have all the pieces needed to assemble the linear system, $K u = f$.
We define a function, `doassemble` to do the assembly, which takes our `cellvalues`,
the sparse matrix and our DofHandler as input arguments. The function returns the
assembled stiffness matrix, and the force vector.

### Solution of the differential equation system
The last step is to solve the system. First we call `doassemble`
to obtain the global stiffness matrix `K` and force vector `f`.
Then, to account for the boundary conditions, we use the `apply!` function.
This modifies elements in `K` and `f` respectively, such that
we can get the correct solution vector `u` by using `\`.

In [ ]:
tspan = (0.0,40e-3);
u0 = Vec{2}((0.0, 1.0))
U0 = interpolate(u0, domain); # vector of vectors
#U = similar(U0)

In [ ]:
for i in numsubdomains(domain):-1:1
    print("i = $i: ")
    
    subdomain = getsubdomain(domain, i)
    Amap = paraboliclinearmap(subdomain)
    U[i] = similar(U0[i])
        
    # Method 1: expmv! from Expokit.jl
    #@time Expokit.expmv!(U[i], tspan[end], Amap, U0[i]; tol=1e-4, norm=expmv_norm, m=30);
    
    # Method 2: direct ODE solution using DifferentialEquations.jl
    prob = ODEProblem((du,u,p,t)->A_mul_B!(du,p[1],u), U0[i], tspan, (Amap,));
    @time sol = solve(prob, CVODE_BDF(linear_solver=:GMRES); saveat=tspan, reltol=1e-4, alg_hints=:stiff)
    U[i] = sol.u[end]
end

In [ ]:
u0 * area(domain.domainboundary)

In [ ]:
integrate(U, domain)

In [ ]:
exp(-tspan[end]*btparams[:R2_lp])

In [ ]:
prob = ODEProblem((du,u,p,t)->A_mul_B!(du,p[1],u), u0, tspan, (Amap,));

In [ ]:
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=100); # penelope: 17.42s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=50); # penelope: 30.09s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=10); # penelope: 103.5s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-8, norm=expmv_norm); # penelope: 53.2s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-6, norm=expmv_norm); # penelope: 44.4s

In [ ]:
#@time sol = solve(prob, CVODE_BDF(linear_solver=:GMRES); saveat=tspan, reltol=1e-8, alg_hints=:stiff); # penelope: 90.21s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:GMRES); saveat=tspan, reltol=1e-4, alg_hints=:stiff); # penelope: 33.44s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:BCG); saveat=tspan, reltol=1e-4, alg_hints=:stiff) # penelope: 53.66s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:TFQMR); saveat=tspan, reltol=1e-4, alg_hints=:stiff) # penelope: 18.99s but low accuracy

In [ ]:
#prob_Ku = ODEProblem(K_mul_u!, u0, tspan, (K,), mass_matrix=M);
#@time sol_Ku = solve(prob_Ku, Rosenbrock23(), saveat=tspan, reltol=1e-4, alg_hints=:stiff) #DNF
#@time sol_Ku = solve(prob_Ku, Rodas4(), saveat=tspan, reltol=1e-4, alg_hints=:stiff) #DNF

In [ ]:
@show norm(sol.u[end] - u)/maximum(abs,u);
@show maximum(sol.u[end] - u)/maximum(abs,u);

### Exporting to VTK
To visualize the result we export the grid and our field `u`
to a VTK-file, which can be viewed in e.g. [ParaView](https://www.paraview.org/).

In [ ]:
vtk_grid("bloch_torrey_equation", dh) do vtk
    vtk_point_data(vtk, dh, u)
end

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*